# Creating the routing tables for the hub-to-hub navigation

Importing Floyd-Warshall algorithm and network

In [4]:
import networkx as nx
from routing import floyd_warshall

graph_raw = nx.read_graphml("resources/hub_network.graphml")
graph_dict = nx.to_dict_of_dicts(graph_raw)

The next cell strips the edge attributes to only consist of the length value

In [5]:
weighted_dict = {}
for u, neighbors in graph_dict.items():
    weighted_dict[u] = {}
    for v, attrs in neighbors.items():
        # Convert length to float or int
        weighted_dict[u][v] = float(attrs['length'])

In [6]:
print(weighted_dict)

{'249412813': {'12859911': 8030.0, '258638252': 64215.0, '1614811288': 77697.0, '380394': 101961.0, '32623218': 121352.0, '30497707': 136089.0, '293976419': 144437.0, '251153043': 144612.0, '309985047': 171841.0}, '495149': {'21070416': 18934.0, '21720607': 27789.0, '495720': 54216.0}, '12450401': {'292021710': 52508.0, '492293': 67325.0, '492650': 70070.0, '717360': 75772.0, '1586387413': 96918.0, '510839': 96210.0, '260441234': 106896.0, '339201635': 109203.0, '1559445838': 113190.0, '1683859971': 140154.0, '487520': 187406.0, '1563579099': 219936.0, '21275562': 234230.0}, '15294804': {'344780298': 8328.0, '399181292': 55104.0}, '15934576': {'251475562': 7663.0, '301510190': 55421.0, '3651328': 80469.0, '264387522': 81122.0, '287141123': 102851.0}, '478778': {'3844504503': 11363.0, '251027655': 39816.0, '21282715': 41317.0, '2568873158': 78507.0, '24941871': 79199.0, '21275372': 83194.0, '2082217695': 96305.0, '2301464427': 103871.0, '299014339': 104814.0, '1189210481': 113589.0, '24

Now to compute the route table via the floyd-warshall algorithm:

In [7]:
dist, next_hop = floyd_warshall(weighted_dict)

In [8]:
print("Next hop from #249412813 to #34215417 is #" + next_hop["249412813"]["34215417"])
print("The distance to the next hop is " + str(dist["249412813"]["34215417"]))

Next hop from #249412813 to #34215417 is #12859911
The distance to the next hop is 251051.0


Defining a function that randomly prints a few shortest distances between hubs:

In [ ]:
import random
import pandas as pd

def show_random_routes(dist, next_hop, num_pairs=10):
    nodes = list(dist.keys())
    
    pairs = []
    while len(pairs) < num_pairs:
        u = random.choice(nodes)
        v = random.choice(nodes)
        if u != v:
            pairs.append((u, v))
    
    data = []
    for u, v in pairs:
        data.append([u, v, next_hop[u][v], dist[u][v]])
    
    df = pd.DataFrame(data, columns=["From", "To", "Next Hop", "Distance"])
    return df


And printing that for 10 random hubs:

In [14]:
df = show_random_routes(dist, next_hop, num_pairs=10)
df

,From,To,Next Hop,Distance
0,4307233413,2270641656,1563579099,353862.0
1,2898305766,29711568,691124331,611590.0
2,324900220,1829167021,129540,141324.0
3,21070416,467063,492809,408847.0
4,824796370,27472715,2008410255,124576.0
5,3435692053,12859911,278603511,621450.0
6,21720607,3152774225,299303109,596724.0
7,5633479,835137888,399181371,332353.0
8,399181292,1618614575,1576877415,260353.0
9,1376884649,1718407877,15934576,543777.0


In [15]:

def routing_table_for_node(node, dist, next_hop):
    """
    Print the routing table for a single node.
    
    Parameters:
    - node: source node (string or int)
    - dist: dictionary of shortest distances from floyd_warshall
    - next_hop: dictionary of next hops from floyd_warshall
    """
    if node not in dist:
        raise ValueError(f"Node '{node}' not found in the graph.")
    
    data = []
    for dest in dist[node]:
        if dest != node:
            data.append([dest, next_hop[node][dest], dist[node][dest]])
    
    df = pd.DataFrame(data, columns=["Destination", "Next Hop", "Distance"])
    df = df.sort_values(by="Destination").reset_index(drop=True)
    return df

In [17]:
routing_table_for_node("21275562", dist, next_hop)

,Destination,Next Hop,Distance
0,1052444818,1658259887,468993.0
1,1052444827,1658259887,462795.0
2,106452248,1668555999,338629.0
3,1089169955,80072760,356915.0
4,1138962686,1658259887,418383.0
...,...,...,...
328,917227939,1658259887,486105.0
329,94722504,1658259887,344718.0
330,95966506,1658259887,511159.0
331,96504569,1658259887,462057.0
